In [1]:
import numpy as np
import pandas as pd

In [8]:
train_df = pd.read_csv('data/training_set_VU_DM.csv', parse_dates=['date_time'])
train_df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [4]:
num_records = train_df.shape[0]
print('Number of records:', num_records)

Number of records: 4958347


In [6]:
num_searches = train_df['srch_id'].nunique()
print('Number of unique searches:', num_searches)

Number of unique searches: 199795


In [10]:
count = 0
for column in train_df.columns:
    if train_df[column].isnull().any():
        count += 1
print('Number of columns with missing values:', count)

Number of columns with missing values: 31


In [11]:
# For each column, print the percentage of missing values
for column in train_df.columns:
    missing = train_df[column].isnull().sum()
    percentage_missing = missing / num_records * 100
    print(f'{column}: {percentage_missing:.2f}% missing')

srch_id: 0.00% missing
date_time: 0.00% missing
site_id: 0.00% missing
visitor_location_country_id: 0.00% missing
visitor_hist_starrating: 94.92% missing
visitor_hist_adr_usd: 94.90% missing
prop_country_id: 0.00% missing
prop_id: 0.00% missing
prop_starrating: 0.00% missing
prop_review_score: 0.15% missing
prop_brand_bool: 0.00% missing
prop_location_score1: 0.00% missing
prop_location_score2: 21.99% missing
prop_log_historical_price: 0.00% missing
position: 0.00% missing
price_usd: 0.00% missing
promotion_flag: 0.00% missing
srch_destination_id: 0.00% missing
srch_length_of_stay: 0.00% missing
srch_booking_window: 0.00% missing
srch_adults_count: 0.00% missing
srch_children_count: 0.00% missing
srch_room_count: 0.00% missing
srch_saturday_night_bool: 0.00% missing
srch_query_affinity_score: 93.60% missing
orig_destination_distance: 32.43% missing
random_bool: 0.00% missing
comp1_rate: 97.58% missing
comp1_inv: 97.39% missing
comp1_rate_percent_diff: 98.10% missing
comp2_rate: 59.17% 

If a visitor has a history of purchasing hotels (value for visitor_hist_starrating, visitor_hist_adr_usd) - it might be advantageous to higher weight their behaviour in our model. Because they are more likely to completely go through the booking process.

**Potential drop candidates**
- srch_query_affinity_score

In [18]:
# print the datatype of the 4th column
train_df = train_df.convert_dtypes()
print(train_df.dtypes['srch_saturday_night_bool'])


train_df.astype({'srch_saturday_night_bool': 'bool'}).dtypes

Int64


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [27]:
import datetime

correct_dtypes = [int, pd.Timestamp, int, int, float, float, int, int, int, float, int, float, float, float, int, float, int, float, int, int, int, int, int, int, bool, float, float, bool, int, int, float, int, int, float, int, int, float, int, int, float, int, int, float, int, int, float, int, int, float, int, int, float]

# turn correct_dtypes into a dictionary with the train_df column names as keys
correct_dtypes = dict(zip(train_df.columns, correct_dtypes))
train_df = train_df.astype(correct_dtypes)
train_df.dtypes

TypeError: dtype '<class 'pandas._libs.tslibs.timestamps.Timestamp'>' not understood